# UZOP - Metabolism-associated molecular clasification of hepatocellular carcinoma

Na ovom projektu cilj je napraviti klasifikator koji na temelju metaboličkih gena klasificira rak jetre.


Poveznica na članak:
https://febs.onlinelibrary.wiley.com/doi/full/10.1002/1878-0261.12639

Poveznica na upute pri izradi projekta:
https://docs.google.com/document/d/1G4L84lbYJ67w2aEUd7nmwxjcNVHjLAubFMcRRYsNY6w/edit


## 1) Priprema podataka
Cilj je raditi analizu nad metaboličkim genima.
Njega je moguće dohvatiti na sljedećoj poveznici.
https://static-content.springer.com/esm/art%3A10.1038%2Fnature10350/MediaObjects/41586_2011_BFnature10350_MOESM321_ESM.xls


Korišteni podatci su TCGA i ICGC. U članku se koristi i GEO skup podataka, ali je u uputama navedeno da se on može koristiti na kraju za validaciju **ako ostane vremena**.

In [1]:
import os
import gzip

import matplotlib.pyplot as plt
import pandas
import pandas as pd
from collections import defaultdict
from scipy.stats import median_abs_deviation
import json
import sksurv
from combat.pycombat import pycombat
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, SVM

In [4]:
METABOLIC_FILE = "../res/41586_2011_BFnature10350_MOESM321_ESM.xls"
def load_metabolic_genes(path: str) -> set:
    return set(pd.read_excel(
        path, sheet_name="All metabolic genes"
    )["Gene Symbol"])
mg = load_metabolic_genes(METABOLIC_FILE)

Za gene postoje anotacije koje sadrže informacije o njihovoj duljini i kojem je identifikacijskom kodu (npr. "ENSG00000176022.4") pridružen koji simbol (npr. "B3GALT6"). Može postojati nekoliko gena s istim simbolom, ali različitim duljinama i identifikacijskim kodovima. Takvi su zanemareni radi jednoznačnosti: kod onih skupova podataka koji sadrže samo simbole nije jasno na koji gen se to odnosi, tj. koju duljinu treba uzeti za normalizaciju.

In [6]:
ANNOTATIONS_FILE = "../res/gencode.v22.annotation.gtf.gz"
def load_ann(path: str, keep_symbols: set, filtering: bool = True) -> tuple:
    id_name_map = {}
    # id_len_map = {}
    name_len_map = defaultdict(list)
    name_id_map = defaultdict(list)
    with gzip.open(path, 'r') as inf:
        for line in inf:
            split_line = line.strip().split(b"\t")
            if len(split_line) < 9:
                continue
            if split_line[2] == b"gene":
                gene_info = split_line[8].strip(b";").split(b"; ")
                # assert gene_info[3].startswith(b"gene_name \"")
                gene_name = str(gene_info[3][11:len(gene_info[3]) - 1], "ascii")
                if not filtering or gene_name in keep_symbols:
                    gene_id = str(gene_info[0][9:len(gene_info[0]) - 1], "ascii")
                    gene_len = int(split_line[4]) - int(split_line[3])
                    name_len_map[gene_name].append(gene_len)
                    name_id_map[gene_name].append(gene_id)
                    id_name_map[gene_id] = gene_name
                    # id_len_map[gene_id] = gene_len

    # Ne zelimo duplikate
    # dropping_names = filter(lambda x: any(it != name_len_map[x][0] for it in name_len_map[x]), name_len_map)
    dropping_names = filter(lambda x: len(name_len_map[x]) > 1, name_len_map)

    for it in list(dropping_names):
        for gene_id in name_id_map[it]:
            # del id_len_map[gene_id]
            del id_name_map[gene_id]
        del name_id_map[it]
        del name_len_map[it]

    name_len_df = pd.DataFrame.from_dict(
        {it: jt[0] for it, jt in name_len_map.items()},
        orient="index", columns=["length"]
    )

    return id_name_map, name_len_df
inm, ld = load_ann(ANNOTATIONS_FILE, mg)

{'ENSG00000176022.4': 'B3GALT6', 'ENSG00000162572.18': 'SCNN1D', 'ENSG00000215790.5': 'SLC35E2', 'ENSG00000008130.14': 'NADK', 'ENSG00000187730.7': 'GABRD', 'ENSG00000149527.16': 'PLCH2', 'ENSG00000157881.12': 'PANK4', 'ENSG00000142606.14': 'MMEL1', 'ENSG00000069424.13': 'KCNAB2', 'ENSG00000097021.18': 'ACOT7', 'ENSG00000162426.13': 'SLC45A1', 'ENSG00000074800.12': 'ENO1', 'ENSG00000131686.13': 'CA6', 'ENSG00000197241.3': 'SLC2A7', 'ENSG00000142583.16': 'SLC2A5', 'ENSG00000049239.11': 'H6PD', 'ENSG00000171612.6': 'SLC25A33', 'ENSG00000171608.14': 'PIK3CD', 'ENSG00000173614.12': 'NMNAT1', 'ENSG00000142657.19': 'PGD', 'ENSG00000116649.8': 'SRM', 'ENSG00000177000.9': 'MTHFR', 'ENSG00000011021.20': 'CLCN6', 'ENSG00000162496.7': 'DHRS3', 'ENSG00000204518.2': 'AADACL4', 'ENSG00000188984.10': 'AADACL3', 'ENSG00000116771.5': 'AGMAT', 'ENSG00000162461.7': 'SLC25A34', 'ENSG00000186510.10': 'CLCNKA', 'ENSG00000184908.16': 'CLCNKB', 'ENSG00000159363.16': 'ATP13A2', 'ENSG00000117118.8': 'SDHB', 'EN

Zatim učitavamo TCGA skup podataka mičući one s identifikacijskim kodovima za koje nije definirano preslikavanje u simbole (nisu metabolički geni ili imaju više simbola pridruženih identifikacijskom kodu). Read count stupac normaliziramo TPM metodom (https://www.rna-seqblog.com/rpkm-fpkm-and-tpm-clearly-explained/). Kako bismo mogli povezati izračunate vrijednosti s kliničkim podatcima kasnije, učitavamo i metapodatke koji imenu datoteke pridružuju identifikacijski kod  pacijenta.

In [11]:
TCGA_FOLDER = "../res/gdc_download_20211208_083143.349897"


def add_name_index(df: pd.DataFrame, id_name_map: dict) -> None:
    df["gene_name"] = df.apply(lambda x: id_name_map[x["gene_id"]], axis=1)
    df.set_index("gene_name", inplace=True)


def remove_nameless_ids(df: pd.DataFrame, id_name_map: dict) -> None:
    df.drop(
        df[[it not in id_name_map for it in df["gene_id"]]].index, inplace=True
    )



def tpm_normalize(df: pd.DataFrame, lengths_df: pd.DataFrame):
    rpk_df = df["raw_read_count"] / lengths_df["length"]
    scaling_factor = rpk_df.count() / 1000000
    df["tpm"] = rpk_df / scaling_factor
    df.drop(columns="raw_read_count", inplace=True)   

def load_meta_tcga(path):
    with open(path, 'r') as inf:
        return {it["file_id"]: it["associated_entities"][0]["case_id"]
                for it in json.load(inf)}

def load_tcga(
        path: str,
        id_name_map: dict,
        patient_case_map: dict,
        length_df: pd.DataFrame
) -> pd.DataFrame:
    patients = pd.DataFrame(index=length_df.index.copy())
    # print(sorted(patients.index))
    # print("# of patients", len(patients.index))
    for patient in os.listdir(path):
        new_path = os.path.join(path, patient)
        if not os.path.isdir(new_path):
            continue
        patient_fn = next(filter(
            lambda x: x.endswith(".gz"),
            os.listdir(new_path)
        ))
        patient_path = os.path.join(new_path, patient_fn)
        df = pd.read_csv(
            patient_path, sep='\t', names=("gene_id", "raw_read_count")
        )
        remove_nameless_ids(df, id_name_map)
        df.dropna(axis=1, inplace=True)
        add_name_index(df, id_name_map)
        df.drop(columns="gene_id", inplace=True)
        tpm_normalize(df, length_df)
        patients[patient_case_map[patient]] = df["tpm"]
    return patients

mt = load_meta_tcga("../res/metadata.cart.2021-12-10.json")
tcga = load_tcga(TCGA_FOLDER, inm, mt, ld)

Na sličan način je učitan icgc skup podataka, ali budući da ni za jednog pacijenta nije zabilježeno više od 2600 relevantnih gena pa je bilo null vrijednosti u tablici (što ne radi s combat algoritmom), nije korištena kasnije.

Postoji mogućnost micanja gena s nepoznatim vrijednostima, ali i to bi značajno smanjilo broj gena s kojima se raspolaže.

Dodatno, svaki pacijent (donor_id) ima dva uzorka pa bi međusobno korelirani podatci (kojih nema u TCGA skupu podataka) mogli utjecati na daljnje izračune.

In [13]:


def filter_by_gene_name(df: pd.DataFrame, keep_symbols: set):
    df.drop(
        df[[it not in keep_symbols for it in df["gene_name"]]].index, inplace=True
    )

ICGC_FILE = "../res/icgc-dataset-1638970984337/exp_seq.tsv.gz"
def load_icgc(path: str, id_name_map: dict, length_df: pd.DataFrame, keep_symbols: set) -> pd.DataFrame:
    patients = pd.DataFrame(index=length_df.index.copy())
    keep_columns = [
        "icgc_sample_id",
        "gene_id",
        "raw_read_count"
    ]
    df = pd.read_csv(
        path,
        sep="\t",
        #  index_col=["icgc_sample_id", "gene_id"],
        usecols=keep_columns
    )
    df.rename(columns={"gene_id": "gene_name"}, inplace=True)
    # df.drop_duplicates(inplace=True, ignore_index=True)
    keep_symbols = set(length_df.index)
    filter_by_gene_name(df, keep_symbols)
    df.dropna(axis=1, inplace=True)
    dfs = [it for it in df.groupby(["icgc_sample_id"])]
    # pd.set_option('display.max_columns', None)
    for i, (patient, df) in enumerate(dfs):
        # for gene_name, df2 in df.groupby(["gene_name"]):
        #     print(df2.head())
        # exit()
        df.set_index("gene_name", inplace=True)
        if len(df.index) > 2600:
            print(patient, len(df.index))
        tpm_normalize(df, length_df)
        if df.isnull().values.any():
            print("patient")
        patients[patient] = df["tpm"]
        if patients.isnull().values.any():
            continue
            print(patient)
            patients.dropna(axis=1, inplace=True)
    # print(len(patients[patients.isnull()].index.tolist()))
    return patients
icgc = load_icgc(ICGC_FILE, inm, ld, mg)

First: False


In [ ]:

TCGA_CLINICAL = "../res/clinical.cart.2021-12-08/clinical.tsv"
RELEVANT_GENES = "../res/mol212639-sup-0006-tables1-s11.xlsx"

def mad(df: pd.DataFrame, threshold: float) -> None:
    df.drop(df.loc[df.apply(
        median_abs_deviation, axis=1, raw=True
    ) <= threshold].index, inplace=True)


# mad(tcga, 0.5)
# cox(tcga, 1.5)

# print(tcga.head())
# icgc.dropna(axis=1, inplace=True)
# print(icgc.iloc[:20])
# total = pd.concat([tcga, icgc], join="inner", axis=1)
# batch = [0] * len(tcga.columns) + [1] * len(icgc.columns)
# data_corrected = pycombat(total, batch)

load_clinical(TCGA_CLINICAL)
# budući da ne koristim icgc dataset, odlučio sam
# nastaviti s relevantnim genima dalje kako bih dobio relevantne
# rezultate
rg = relevant_genes(RELEVANT_GENES)
X = tcga[tcga.index.isin(rg)]
print(len(rg))
print(X)
kmeans = KMeans(3, random_state=42).fit(X)
results = pandas.DataFrame(data=kmeans.labels_, columns=["cluster"], index=X.index).groupby("cluster")

reduced = PCA(2, random_state=42).fit_transform(X)
# print("r", reduced)
plt.figure()
for name, result in results:
    # print(name, X.index)
    # print(reduced[X.index.isin(result.index)])
    plt.scatter(*reduced[X.index.isin(result.index)].T)
plt.show()

classificator = SVM().fit(X, results.labels_)

Izgleda da ima outliera koji poremete odabir grupa. Zato klasifikator ne radi. Njih se može maknuti trimmanjem onih koji odskaču previše, ali svejedno klasifikator nije najbolji.